In [2]:
pip install pyodbc pandas mlxtend 

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pyodbc

# Define the connection string
conn_str = (
    r'DRIVER={ODBC Driver 17 for SQL Server};'
    r'SERVER=LAPTOP-ATISN7C4;'
    r'DATABASE=CompanyX;'
    r'Trusted_Connection=yes;'
)

print(conn_str)

# Establish the connection
conn = pyodbc.connect(conn_str)

# Create a cursor object
cursor = conn.cursor()

# Test the connection by executing a simple query
cursor.execute("SELECT @@VERSION")

# Fetch and print the result
row = cursor.fetchone()
print(row[0])

# Close the cursor and connection


DRIVER={ODBC Driver 17 for SQL Server};SERVER=LAPTOP-ATISN7C4;DATABASE=CompanyX;Trusted_Connection=yes;
Microsoft SQL Server 2022 (RTM) - 16.0.1000.6 (X64) 
	Oct  8 2022 05:58:25 
	Copyright (C) 2022 Microsoft Corporation
	Developer Edition (64-bit) on Windows 10 Home Single Language 10.0 <X64> (Build 22631: ) (Hypervisor)



In [4]:
import pandas as pd

In [5]:
querySalesFact = "SELECT * FROM dbo.SalesFact"
df_SalesFact = pd.read_sql(querySalesFact, conn)

queryProductDim = "Select * from dbo.ProductDim"
queryLocationDim = "Select * from dbo.LocationDim"
queryDateDim = "Select * from dbo.DateDim"
queryPromotionDim = "Select * from dbo.PromotionDim"
queryTerritoryDim = "Select * from dbo.TerritoryDim"
queryCustomerDim = "Select * from dbo.CustomerDim"

df_ProductDim = pd.read_sql(queryProductDim,conn)
df_LocationDim = pd.read_sql(queryLocationDim,conn)
df_DateDim = pd.read_sql(queryDateDim,conn)
df_PromotionDim = pd.read_sql(queryPromotionDim,conn)
df_TerritoryDim = pd.read_sql(queryTerritoryDim,conn)
df_CustomerDim = pd.read_sql(queryCustomerDim,conn)


C:\Users\Admin\AppData\Local\Temp\ipykernel_3376\2195563624.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_SalesFact = pd.read_sql(querySalesFact, conn)
C:\Users\Admin\AppData\Local\Temp\ipykernel_3376\2195563624.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ProductDim = pd.read_sql(queryProductDim,conn)
C:\Users\Admin\AppData\Local\Temp\ipykernel_3376\2195563624.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_LocationDim = pd.read_sql(queryLocationDim,conn)
C:\Users\Admin\AppData\Local\Temp\ipyker

In [6]:
import numpy as np 
from mlxtend.frequent_patterns import apriori, association_rules 

df_extracted = df_SalesFact[['TransactionID', 'ProductID', 'QuantitySold']].fillna(0)
df_extracted = (df_extracted.groupby(['TransactionID', 'ProductID'])['QuantitySold']
    .sum().unstack().reset_index().fillna(0) 
    .set_index('TransactionID')) 

df_extracted.head()

ProductID,707,708,709,710,711,712,713,714,715,716,...,989,990,991,992,993,994,996,997,998,999
TransactionID,,,,,,,,,,,,,,,,,,,,,
43659,0.0,0.0,24.0,0.0,20.0,8.0,0.0,9.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43660,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43661,0.0,25.0,0.0,0.0,10.0,16.0,0.0,0.0,16.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
43663,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1):
        return 1

df_extracted = df_extracted.applymap(hot_encode)
df_extracted.fillna(0, inplace=True)

df_extracted.fillna(0)
frq_items = apriori(df_extracted, min_support = 0.01, use_colnames = True) 
  
rules = association_rules(frq_items, metric ="lift", min_threshold = 0, num_itemsets = len(frq_items)) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 
print(rules.head()) 

C:\Users\Admin\AppData\Local\Temp\ipykernel_3376\4057656083.py:7: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_extracted = df_extracted.applymap(hot_encode)
c:\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


           antecedents consequents  antecedent support  consequent support  \
14395  (864, 707, 708)  (883, 884)            0.010025              0.0121   
15355  (864, 707, 711)  (883, 884)            0.010025              0.0121   
16165  (864, 707, 715)  (883, 884)            0.010057              0.0121   
16375  (707, 876, 715)  (883, 884)            0.010057              0.0121   
16555  (864, 707, 876)  (883, 884)            0.010025              0.0121   

        support  confidence       lift  representativity  leverage  \
14395  0.010025         1.0  82.646438               1.0  0.009903   
15355  0.010025         1.0  82.646438               1.0  0.009903   
16165  0.010057         1.0  82.646438               1.0  0.009935   
16375  0.010057         1.0  82.646438               1.0  0.009935   
16555  0.010025         1.0  82.646438               1.0  0.009903   

       conviction  zhangs_metric   jaccard  certainty  kulczynski  
14395         inf       0.997904  0.828496

In [8]:
# Create a dictionary to map ProductID to ProductName
product_id_to_name = df_ProductDim.set_index('ProductID')['ProductName'].to_dict()

# Function to replace ProductID with ProductName in itemsets
def replace_ids_with_names(itemset):
    return frozenset([product_id_to_name.get(item, item) for item in itemset])

# Apply the function to the antecedents and consequents columns
rules['antecedents'] = rules['antecedents'].apply(replace_ids_with_names)
rules['consequents'] = rules['consequents'].apply(replace_ids_with_names)

rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
14395,"(Sport-100 Helmet, Black, Sport-100 Helmet, Re...","(Short-Sleeve Classic Jersey, XL, Short-Sleeve...",0.010025,0.012100,0.010025,1.00000,82.646438,1.0,0.009903,inf,0.997904,0.828496,1.000000,0.914248
15355,"(Sport-100 Helmet, Red, Classic Vest, S, Sport...","(Short-Sleeve Classic Jersey, XL, Short-Sleeve...",0.010025,0.012100,0.010025,1.00000,82.646438,1.0,0.009903,inf,0.997904,0.828496,1.000000,0.914248
16165,"(Long-Sleeve Logo Jersey, L, Sport-100 Helmet,...","(Short-Sleeve Classic Jersey, XL, Short-Sleeve...",0.010057,0.012100,0.010057,1.00000,82.646438,1.0,0.009935,inf,0.997936,0.831135,1.000000,0.915567
16375,"(Long-Sleeve Logo Jersey, L, Hitch Rack - 4-Bi...","(Short-Sleeve Classic Jersey, XL, Short-Sleeve...",0.010057,0.012100,0.010057,1.00000,82.646438,1.0,0.009935,inf,0.997936,0.831135,1.000000,0.915567
16555,"(Hitch Rack - 4-Bike, Sport-100 Helmet, Red, C...","(Short-Sleeve Classic Jersey, XL, Short-Sleeve...",0.010025,0.012100,0.010025,1.00000,82.646438,1.0,0.009903,inf,0.997904,0.828496,1.000000,0.914248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,(Water Bottle - 30 oz.),"(Sport-100 Helmet, Red, Half-Finger Gloves, M)",0.149666,0.019251,0.010025,0.06698,3.479270,1.0,0.007143,1.051155,0.838004,0.063090,0.048665,0.293855
1189,(Water Bottle - 30 oz.),"(Sport-100 Helmet, Black, Half-Finger Gloves, M)",0.149666,0.019347,0.010025,0.06698,3.462046,1.0,0.007129,1.051052,0.836323,0.063052,0.048572,0.292566
3294,(Water Bottle - 30 oz.),"(Long-Sleeve Logo Jersey, L, Sport-100 Helmet,...",0.149666,0.024295,0.010025,0.06698,2.756898,1.0,0.006388,1.045748,0.749439,0.061149,0.043747,0.239797
3044,(Water Bottle - 30 oz.),"(Sport-100 Helmet, Red, Sport-100 Helmet, Blac...",0.149666,0.024359,0.010025,0.06698,2.749672,1.0,0.006379,1.045680,0.748318,0.061125,0.043685,0.239256


In [9]:
# Filter rules with only one element in both antecedents and consequents
filtered_rules = rules[rules['antecedents'].apply(lambda x: len(x) == 1) & rules['consequents'].apply(lambda x: len(x) == 1)]
filtered_rules['antecedents'] = filtered_rules['antecedents'].apply(replace_ids_with_names)
filtered_rules['consequents'] = filtered_rules['consequents'].apply(replace_ids_with_names)

print(filtered_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])
extractedRules = filtered_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]



                 antecedents                       consequents   support  \
125      (Women's Tights, L)                    (AWC Logo Cap)  0.013217   
326      (Women's Tights, S)               (Women's Tights, L)  0.012611   
173      (Women's Tights, L)      (Long-Sleeve Logo Jersey, L)  0.013153   
82     (Men's Bib-Shorts, M)          (Sport-100 Helmet, Blue)  0.010472   
123      (Women's Tights, S)                    (AWC Logo Cap)  0.012387   
..                       ...                               ...       ...   
365  (Water Bottle - 30 oz.)                 (Classic Vest, M)  0.011908   
389  (Water Bottle - 30 oz.)  (Short-Sleeve Classic Jersey, S)  0.011685   
384  (Water Bottle - 30 oz.)           (Fender Set - Mountain)  0.010440   
335  (Water Bottle - 30 oz.)           (Half-Finger Gloves, S)  0.010344   
159  (Water Bottle - 30 oz.)      (Long-Sleeve Logo Jersey, M)  0.010152   

     confidence       lift  
125    0.938776   8.699783  
326    0.936019  66.482816  


C:\Users\Admin\AppData\Local\Temp\ipykernel_3376\3101147571.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rules['antecedents'] = filtered_rules['antecedents'].apply(replace_ids_with_names)
C:\Users\Admin\AppData\Local\Temp\ipykernel_3376\3101147571.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_rules['consequents'] = filtered_rules['consequents'].apply(replace_ids_with_names)


In [10]:

# Example: Dynamically create the table
table_name = 'pairRule'
columns = ', '.join([f"[{col}] NVARCHAR(MAX)" for col in extractedRules.columns])
create_table_query = f"CREATE TABLE {table_name} ({columns});"

try:
    cursor.execute(f"DROP TABLE IF EXISTS {table_name};")  # Drop the table if it exists
    cursor.execute(create_table_query)
    conn.commit()
except Exception as e:
    print(f"Error creating table: {e}")


In [11]:
# Convert the StartDate and EndDate columns to datetime if they are not already
df_PromotionDim['StartDate'] = pd.to_datetime(df_PromotionDim['StartDate'])
df_PromotionDim['EndDate'] = pd.to_datetime(df_PromotionDim['EndDate'])

# Create a dictionary to store dataframes for each promotion period
promotion_dfs = {}

# Iterate through each promotion
for _, row in df_PromotionDim.iterrows():
    promotion_id = row['PromotionID']
    start_date = row['StartDate']
    end_date = row['EndDate']
    
    # Filter the SalesFact dataframe for the given promotion period
    promotion_df = df_SalesFact[(df_SalesFact['PromotionID'] == promotion_id) & 
                                (df_SalesFact['ModifiedDate'] >= start_date) & 
                                (df_SalesFact['ModifiedDate'] <= end_date)]
    
    # Store the dataframe in the dictionary
    promotion_dfs[promotion_id] = promotion_df

# Create a dataframe for transactions with no promotion
no_promotion_df = df_SalesFact[df_SalesFact['PromotionID'] == 1]

# Display the dataframes
for promotion_id, promotion_df in promotion_dfs.items():
    print(f"Promotion ID: {promotion_id}")
    print(promotion_df.head())

print("No Promotion DataFrame:")
print(no_promotion_df.head())

Promotion ID: 1
    SalesFactID  TransactionID  ProductID  CustomerID  TerritoryID  \
94           95          43886        754       29698            6   
95           96          43917        754       29885            1   
96           97          43917        761       29885            1   
97           98          44117        754       29581            2   
98           99          44117        761       29581            2   

    PromotionID  DateID  QuantitySold  UnitPrice  TotalSaleAmount  \
94            1     411           2.0   874.7940        1749.5880   
95            1     411           5.0   874.7940        4373.9700   
96            1     411           1.0   419.4589         419.4589   
97            1      26           2.0   874.7940        1749.5880   
98            1      26           1.0   419.4589         419.4589   

    DiscountApplied  LocationID            ModifiedDate  
94              0.0          60 2014-02-08 10:01:36.827  
95              0.0          60 

In [12]:
def process_dataframe(df):
    df_extracted = df[['TransactionID', 'ProductID', 'QuantitySold']].fillna(0)
    df_extracted = (df_extracted.groupby(['TransactionID', 'ProductID'])['QuantitySold']
                    .sum().unstack().reset_index().fillna(0)
                    .set_index('TransactionID'))
    return df_extracted

# Apply the function to each dataframe in promotion_dfs
for promotion_id in promotion_dfs:
    promotion_dfs[promotion_id] = process_dataframe(promotion_dfs[promotion_id])

# Apply the function to no_promotion_df
no_promotion_df_processed = process_dataframe(no_promotion_df)

In [13]:
for promotion_id in promotion_dfs:
    promotion_dfs[promotion_id] = promotion_dfs[promotion_id].applymap(hot_encode)


C:\Users\Admin\AppData\Local\Temp\ipykernel_3376\7612559.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  promotion_dfs[promotion_id] = promotion_dfs[promotion_id].applymap(hot_encode)


In [14]:
no_promotion_df_processed = no_promotion_df_processed.applymap(hot_encode)

C:\Users\Admin\AppData\Local\Temp\ipykernel_3376\1819978960.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  no_promotion_df_processed = no_promotion_df_processed.applymap(hot_encode)


In [15]:
frq_nopromotion_items = apriori(no_promotion_df_processed, min_support = 0.03, use_colnames=True)
nonpromotion_rules = association_rules(frq_nopromotion_items, metric = "lift", min_threshold=0, num_itemsets = len(frq_nopromotion_items))
nonpromotion_rules = nonpromotion_rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

frq_promotion_items = {}
promotion_rules = {}
promotion_rules_id = []
for promotion_id in promotion_dfs:
    frq_promotion_items[promotion_id] = apriori(promotion_dfs[promotion_id], min_support=0.03, use_colnames=True)
    if (len(frq_promotion_items[promotion_id]) > 0):
        promotion_rules_id.append(promotion_id)
        promotion_rules[promotion_id] = association_rules(frq_promotion_items[promotion_id], metric = "lift", min_threshold=0, num_itemsets = len(frq_promotion_items[promotion_id]))
        promotion_rules[promotion_id].sort_values(['confidence', 'lift'], ascending =[False, False]) 

c:\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
c:\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
c:\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
c:\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their

In [16]:
for id in promotion_rules_id :
    print(promotion_rules[id])

   antecedents consequents  antecedent support  consequent support   support  \
0        (712)       (707)            0.107908            0.098362  0.030425   
1        (707)       (712)            0.098362            0.107908  0.030425   
2        (712)       (708)            0.107908            0.095936  0.031478   
3        (708)       (712)            0.095936            0.107908  0.031478   
4        (712)       (711)            0.107908            0.098586  0.032213   
5        (711)       (712)            0.098586            0.107908  0.032213   
6        (712)       (715)            0.107908            0.052102  0.037353   
7        (715)       (712)            0.052102            0.107908  0.037353   
8        (712)       (870)            0.107908            0.149666  0.032532   
9        (870)       (712)            0.149666            0.107908  0.032532   
10       (870)       (871)            0.149666            0.064649  0.054018   
11       (871)       (870)            0.

In [18]:
# Create an empty list to store the results
results = []

# Iterate through each promotion rule
for promotion_id in promotion_rules_id:
    promotion_rule = promotion_rules[promotion_id]
    
    # Iterate through each row in the promotion rule
    for _, row in promotion_rule.iterrows():
        antecedents = row['antecedents']
        consequents = row['consequents']
        confidence = row['confidence']
        lift = row['lift']
        
        # Find the corresponding row in non_promotion_rules
        match = nonpromotion_rules[(nonpromotion_rules['antecedents'] == antecedents) & 
                                   (nonpromotion_rules['consequents'] == consequents)]
        
        if not match.empty:
            non_promotion_confidence = match.iloc[0]['confidence']
            non_promotion_lift = match.iloc[0]['lift']
            
            # Check if the confidence in promotion_rules is higher
            if confidence > non_promotion_confidence :
                results.append({
                    'antecedents': antecedents,
                    'consequents': consequents,
                    'promotion_id': promotion_id,
                    'promotion_confidence': confidence,
                    'non_promotion_confidence': non_promotion_confidence,
                    'confidence_difference': confidence - non_promotion_confidence
                })

# Convert the results to a DataFrame
df_higher_confidence = pd.DataFrame(results)

# Display the DataFrame
print(df_higher_confidence)

  antecedents consequents  promotion_id  promotion_confidence  \
0       (712)       (870)             2              0.301479   
1       (870)       (712)             2              0.217363   
2       (712)       (707)             3              0.281953   
3       (712)       (870)             3              0.301479   
4       (870)       (712)             3              0.217363   
5       (712)       (870)             4              0.301479   

   non_promotion_confidence  confidence_difference  
0                  0.301479           5.551115e-17  
1                  0.217363           5.551115e-17  
2                  0.281953           5.551115e-17  
3                  0.301479           5.551115e-17  
4                  0.217363           5.551115e-17  
5                  0.301479           5.551115e-17  


In [33]:
print(df_SalesFact[1:30])

    SalesFactID  TransactionID  ProductID  CustomerID  TerritoryID  PromotionID  DateID  QuantitySold  UnitPrice  TotalSaleAmount  DiscountApplied  LocationID            ModifiedDate
1             2          49485        794       29639            6            4    1089           3.0  1308.9375        3926.8125              0.0          60 2014-02-08 10:01:36.827
2             3          49518        794       29966            2            4    1089           1.0  1308.9375        1308.9375              0.0          60 2014-02-08 10:01:36.827
3             4          49799        794       23994            9            4     939           1.0  2181.5625        2181.5625              0.0          60 2014-02-08 10:01:36.827
4             5          48349        794       29486            3            4     293           2.0  1308.9375        2617.8750              0.0          60 2014-02-08 10:01:36.827
5             6          47426        794       29883            3            4    10

In [49]:
df_extracted = df_SalesFact[['TransactionID', 'ProductID', 'QuantitySold', 'ModifiedDate']].fillna(0)
df_extracted['ProductPromotionID'] = df_SalesFact['ProductID'].astype(str) + '_' + df_SalesFact['PromotionID'].astype(str)

df_extracted.where(df_extracted['ProductID'] == 877).dropna()

,TransactionID,ProductID,QuantitySold,ModifiedDate,ProductPromotionID
3888,51739.0,877.0,10.0,2014-02-08 10:01:36.827,877_1
5184,51739.0,877.0,10.0,2014-02-08 10:01:36.827,877_3
5194,53779.0,877.0,1.0,2014-02-08 10:01:36.827,877_3
5195,54029.0,877.0,1.0,2014-02-08 10:01:36.827,877_3
5199,51120.0,877.0,3.0,2014-02-08 10:01:36.827,877_3
...,...,...,...,...,...
424350,63094.0,877.0,1.0,2014-02-08 10:01:36.827,877_3
424352,63179.0,877.0,7.0,2014-02-08 10:01:36.827,877_3
424354,60805.0,877.0,1.0,2014-02-08 10:01:36.827,877_3
424359,61189.0,877.0,5.0,2014-02-08 10:01:36.827,877_3


In [55]:
df_extracted = df_SalesFact[['TransactionID', 'ProductID', 'QuantitySold', 'ModifiedDate']].fillna(0)
df_extracted['ProductPromotionID'] = df_SalesFact['ProductID'].astype(str) + '_' + df_SalesFact['PromotionID'].astype(str)


df_extracted = (df_extracted.groupby(['TransactionID', 'ProductPromotionID'])['QuantitySold']
    .sum().unstack().reset_index().fillna(0) 
    .set_index('TransactionID')) 

df_extracted.head()
df_extracted = df_extracted.applymap(hot_encode)
df_extracted.fillna(0, inplace=True)

frq_items = apriori(df_extracted, min_support = 0.03, use_colnames = True)


rules = association_rules(frq_items, metric ="lift", min_threshold = 0, num_itemsets = len(frq_items))
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])

C:\Users\Admin\AppData\Local\Temp\ipykernel_3376\4183514348.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_extracted = df_extracted.applymap(hot_encode)
c:\Python310\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [56]:
frq_items

,support,itemsets
0,0.098362,(707_1)
1,0.098362,(707_11)
2,0.098362,(707_2)
3,0.098362,(707_3)
4,0.098362,(707_8)
...,...,...
2108,0.037353,"(715_4, 715_1, 715_3, 715_2, 712_1, 712_3, 712..."
2109,0.032532,"(870_1, 870_3, 712_1, 712_3, 712_4, 712_2, 870..."
2110,0.030425,"(712_2, 707_2, 707_1, 712_1, 712_3, 707_8, 712..."
2111,0.031478,"(708_1, 708_8, 708_2, 712_1, 712_3, 712_4, 712..."


In [58]:
def clean_itemset(itemset):
    # Split items into prefix and suffix and group by prefix
    prefix_map = {}
    for item in itemset:
        prefix, suffix = item.split('_')
        suffix = int(suffix)
        if prefix not in prefix_map or suffix < prefix_map[prefix][1]:
            prefix_map[prefix] = (item, suffix)
    
    # Extract only the items with the smallest suffix per prefix
    return {v[0] for v in prefix_map.values()}

# Apply the function to clean the itemsets
frq_items['itemsets'] = frq_items['itemsets'].apply(clean_itemset)



In [61]:

rules = association_rules(frq_items, metric ="lift", min_threshold = 0, num_itemsets = len(frq_items))
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])

print(rules)

    antecedents consequents  antecedent support  consequent support   support  confidence      lift  representativity  leverage  conviction  zhangs_metric   jaccard  certainty  kulczynski
186     (872_1)     (870_1)            0.054656            0.149666  0.048559    0.888435  5.936100               1.0  0.040378    7.621839       0.879616  0.311744   0.868798    0.606440
190     (872_1)     (870_2)            0.054656            0.149666  0.048559    0.888435  5.936100               1.0  0.040378    7.621839       0.879616  0.311744   0.868798    0.606440
194     (872_1)     (870_3)            0.054656            0.149666  0.048559    0.888435  5.936100               1.0  0.040378    7.621839       0.879616  0.311744   0.868798    0.606440
198     (872_1)     (870_4)            0.054656            0.149666  0.048559    0.888435  5.936100               1.0  0.040378    7.621839       0.879616  0.311744   0.868798    0.606440
185     (871_1)     (870_1)            0.064649            0

In [67]:
def remove_no_promotion(rule):
    for id, row in rule.iterrows():
        antecedents = list(row['antecedents'])[0]
        consequents = list(row['consequents'])[0]
        _, a_suffix = antecedents.split('_')
        _, c_suffix = consequents.split('_')
        if a_suffix == '1' and c_suffix == '1':
            rule.drop(index=id, inplace=True)

# Remove rules with no promotion
remove_no_promotion(rules)

print(rules)

        

    antecedents consequents  antecedent support  consequent support   support  confidence      lift  representativity  leverage  conviction  zhangs_metric   jaccard  certainty  kulczynski
190     (872_1)     (870_2)            0.054656            0.149666  0.048559    0.888435  5.936100               1.0  0.040378    7.621839       0.879616  0.311744   0.868798    0.606440
194     (872_1)     (870_3)            0.054656            0.149666  0.048559    0.888435  5.936100               1.0  0.040378    7.621839       0.879616  0.311744   0.868798    0.606440
198     (872_1)     (870_4)            0.054656            0.149666  0.048559    0.888435  5.936100               1.0  0.040378    7.621839       0.879616  0.311744   0.868798    0.606440
189     (871_1)     (870_2)            0.064649            0.149666  0.054018    0.835556  5.582787               1.0  0.044342    5.170948       0.877615  0.336985   0.806612    0.598239
193     (871_1)     (870_3)            0.064649            0

1743.0200000000004